# Calculating index jump rate in a pooled NGS run with dual indexes
This tool is used to calculate the rate at which index jumps occur for a single sequencing run. Though this calculation process should be reproducible across many use cases, there are a few key requirements which must be met for it to be accurate.

1. Dual-indexed reads (i.e. an index or barcode placed on both the forward and reverse end), which have been demultiplexed using QIIME2


2. Use of two or more dual-index pairs 


3. **(OPTIONAL, BUT RECOMENDED)** 'Calibrator tags', or the use of at least one forward and one reverse index pair exclusively with each other. While one pair of calibraotr tags is recomended for an accurate estimate, we provide support for specifying multiple pairs (increased accuracy), or no pairs at all for calcuating based purely from false reads (decreased accuracy).

In [1]:
import pathlib
import shutil
import tempfile
import math
import os
import pandas as pd
from IPython.display import display_html
from qiime2 import Artifact, Metadata
from qiime2.plugins.demux.visualizers import summarize

def extract_tsvs(viz, dest):
    '''Extracts useful TSV from QIIME artifact file'''
    with tempfile.TemporaryDirectory() as temp:
        viz.export_data(temp)
        temp_pathlib = pathlib.Path(temp)
        for file in temp_pathlib.iterdir():
            if file.suffix == '.tsv':
                shutil.copy(file, dest)

 ### Read in our sample map containing all possible 20bp indexes
 
 In the True_False column, 1s indicate a true read combo, where 0s 
 indicate a false read combo / one that was not used for a any samples in the library pool 

In [2]:
metadata = Metadata.load('Sample_Map_All_Indexes.txt')
metadata_df = pd.read_csv('Sample_Map_All_Indexes.txt', sep='\t')
metadata_df

,#SampleID,BarcodeSequence,Fwd_Index,Rev_Index,True_False
0,F01R11,CTAAGGTAACTCCTCGAATC,1,11,1
1,F01R12,CTAAGGTAACTAGGTGGTTC,1,12,0
2,F01R13,CTAAGGTAACTCTAACGGAC,1,13,0
3,F01R14,CTAAGGTAACTTGGAGTGTC,1,14,0
4,F01R15,CTAAGGTAACTCTAGAGGTC,1,15,0
...,...,...,...,...,...
95,F10R16,CTGACCGAACTCTGGATGAC,10,16,1
96,F10R17,CTGACCGAACTCTATTCGTC,10,17,1
97,F10R18,CTGACCGAACAGGCAATTGC,10,18,1
98,F10R19,CTGACCGAACTTAGTCGGAC,10,19,1


### Loading and summarizing demultiplexed data with QIIME2 API 
This summarization shows the number of reads present in each of our 100 index combinations that are possible within our sampel pool. In our sample data, we see that this read depth is even among all our True reads, except for two outliers (F10R15 and F10R12), and that our False combos have between 0 and 5 reads. 

We can then extract this per sample read count, load it in as a dataframe, and take subsets of this data frame to prepare for calculating the index jump rate for our sample pool using our calibrator tags. 

In this example we have reserved the forward tag 1 (F01) and the reverse tag 11 (R11) as our calibrator tags for ease. When using the the provided python script, you are able to specify whatever tags were used in your own unique library preparation.

In [3]:
sequences = Artifact.load('demultiplexed_seqs.qza')
demux_summary = summarize(sequences)
demux_viz = demux_summary.visualization
demux_viz

<visualization: Visualization uuid: e621f44f-f789-40bd-9276-b62d070cc92b>

In [4]:
# Extracting a per-sample read count from QIIME Artifact as dataframe
os.makedirs('tsvs', exist_ok=True)
extract_tsvs(demux_viz, 'tsvs')
df = pd.read_csv("./tsvs/per-sample-fastq-counts.tsv", sep="\t")

# Sum all read counts for pool total
total_reads = df['forward sequence count'].sum()
print(f'Total Reads: {total_reads}')

Total Reads: 481749


In [5]:
# Add Fwd and Rev columns for individual ID sorting

df['Fwd'] = df['sample ID'].str[1:3]
df['Rev'] = df['sample ID'].str[4:6]

# Reindex and add the True_False metadata column

df = df.sort_values(['sample ID'])
df.reset_index(inplace=True)
df['True_False'] = metadata_df['True_False']

# Grab F1 and R11 data frames for tag jump calculations

F1  = df[df['Fwd'] == '01']
R11 = df[df['Rev'] == '11']

df1_styler = F1.style.set_table_attributes("style='display:inline'").set_caption('F1 - Forward Calibrator Tag')
df2_styler = R11.style.set_table_attributes("style='display:inline'").set_caption('R11 - Reverse Calibrator Tag')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

,index,sample ID,forward sequence count,Fwd,Rev,True_False
0,0,F01R11,6250,01,11,1
1,83,F01R12,1,01,12,0
2,88,F01R13,0,01,13,0
3,89,F01R14,0,01,14,0
4,84,F01R15,1,01,15,0
5,90,F01R16,0,01,16,0
6,91,F01R17,0,01,17,0
7,87,F01R18,0,01,18,0
8,93,F01R19,0,01,19,0
9,92,F01R20,0,01,20,0


### Calculating the Average Jump Rate of our two calibrator tags 
The jump rate of each calibrator tag is calculated by taking the false reads with that tag, and dividing them by the total number of that tag present in the pool. 

In this instance a false read is any read which has an F01 tag and not an R11 tag, or any read with an R11 tag and not an F01. The jump rate for the F01 tag and R11 tag are then averaged as a best estimate for the overall rate at which any index jumps occur.

This method of calculating jump rate based off of calibrator tags offers increased accuracy when compared to calculating jump rate based off of just the total false reads within a pool. 

In using calibrator tags we are able to detect every instance in which a tag jump occured in regards to a single index tag (barring any jumps that don't change sample assignment). In calculating this metric using non-calibrator tags, we lose this level of clarity. For example, if in the above example F01R12 and F01R15 were both True samples, we would not be able to flag the jumps which occured in these samples as false, and would conclude that no jumps occured within our pool. This example, while extreme, demonstrates how with each loss of a calibrator pair, our estimate of the jump rates becomes less exact. 

In [6]:
# Calculating Average jump rate

# Grab all false F1 reads
F1_False = F1[F1["True_False"] == 0]

# Calculate proportion of all F1 reads which are false 
F1_jump_rate = (F1_False['forward sequence count'].sum())/(F1['forward sequence count'].sum())

# Repeat for R11 
R11_False = R11[R11["True_False"] == 0]
R11_jump_rate = (R11_False['forward sequence count'].sum())/(R11['forward sequence count'].sum())

# Calculate the jump rate for any given combination of index tags 
jump_rate = (F1_jump_rate + R11_jump_rate) / 2
print(f'F01: {F1_jump_rate}')
print(f'R11: {R11_jump_rate}')
print(f'Average jump rate: {jump_rate}')

F01: 0.0003198976327575176
R11: 0.00047976971053894133
Average jump rate: 0.0003998336716482295


### Estimating false reads in each sample based on calculated jump rate 
To calculate the number of index jumps expected to occur we need to calculate the amount of jumps we expect to happen on both the reverse and the forward end based on relative tag frequencies and sum them. 

For example, with the dual-tag F10R17, a false read could occur due to an F10 tag jumping on to any R17 read, or an R17 tag jumping on to any F10 read. Each of those amounts are going to be dependent on the total number of extraneous F10 and R17 tags in the pooled sample. 

Extraneous F10s (F10 tags not related to the F10R17 sample): 31,735

Extraneous R17s (R17 tags not related to the F10R17 sample): 43,755

Because of this difference in extraneous F10 tags and R12 tags, we cannot simply multiply the number of F10R17 reads by the jump rate\*2 , and instead we need to account for these different abundances. 

To do this, we multiply the total number of F10 tags by the average jump rate to give us how many F10 tags we expect to jump. Then we multiply this by the proportion of extraneous R17 reads in the pool, giving the amount of F10 jumps we expect to hit a novel R17 read. The same is then done for R17 to F10. The two counts are then added together to give the total expected amount of false reads for the F10R17 dual-index

Below, we compare how to samples with equal read counts (F04R14 and F10R17) have different amounts of expected false reads within them due to differential abundances of extraneous F04, F14, F10, and R17 tags.

In [7]:
#Prepping lists for calculations

Indexes    = df['sample ID'].tolist()
Seq_counts = df['forward sequence count'].tolist()

Fwds       = df['Fwd'].tolist()
Fwds       = list(set(Fwds))
Fwds.sort()

Revs    = df['Rev'].tolist()
Revs    = list(set(Revs))
Revs.sort()

In [8]:
#Calculating each Indexes expected # of False sequences

indexes_exp_false = [] 
total_read_count = df['forward sequence count'].sum()
count = 0

for i in Fwds:
    temp_df = df[df['Fwd'] == i]
    
    # No. of reads with i FWD index
    i_seqs = temp_df['forward sequence count'].sum()
    # No. of reads with i FWD index and NOT j REV index
    i_seqs_no_j = i_seqs - Seq_counts[count]
    
    for j in Revs:
        temp_df2 = df[df['Rev'] == j]
        
        # No. of reads with j REV index
        j_seqs = temp_df2['forward sequence count'].sum()
        # No. of reads with j REV index and NOT i FWD index
        j_seqs_no_i = j_seqs-Seq_counts[count]
        
        # No. of j reads with an expected false i index
        ## jump rate for i tags
        i_jumps = i_seqs * jump_rate
        ## number of jumping i tags we expect to hit an extraneous j read
        i_to_j = i_jumps * (j_seqs_no_i / total_read_count)
        
        # No. of i reads with an expected false j index
        ## jump rate for j tags
        j_jumps = j_seqs * jump_rate
        ## number of jumping j tags we expect to hit an extraneous i read
        j_to_i = j_jumps * (i_seqs_no_j / total_read_count)
        
        # Sum for total i-j pair false read count
        indiv_tag_jump_count = math.ceil((i_to_j + j_to_i))
        
        # Append to list for data table output
        to_add = (Indexes[count], indiv_tag_jump_count)
        indexes_exp_false.append(to_add)
        count += 1    
        
        if i == '04' and j == '14':
            print('Here we see how despite equal read counts between two samples, the differential ' \
                  'abundances of extraneous \ntags result in different expected false reads within that sample\n')
            
            print('Calculating F04R14 Index Jumps')
            print(f'Total F04R14 reads: {Seq_counts[count]}')
            print(f'Total Reads with F04 and NOT R14:{i_seqs_no_j}')
            print(f'Total Reads with R14 and NOT F04:{j_seqs_no_i}')
            print(f'Expected # of Seqs where F04 jumped to an R14:{i_to_j}')
            print(f'Expected # of Seqs where R14 jumped to an F04:{j_to_i}')
            print(f'Expected # of FALSE F04R14 reads:{indiv_tag_jump_count}\n')
            
        if i == '10' and j == '17':
            print('Calculating F10R17 Index Jumps')
            print(f'Total F10R17 reads: {Seq_counts[count]}')
            print(f'Total Reads with F10 and NOT R17:{i_seqs_no_j}')
            print(f'Total Reads with R15 and NOT F07:{j_seqs_no_i}')
            print(f'Expected # of Seqs where F07 jumped to an R15:{i_to_j}')
            print(f'Expected # of Seqs where R15 jumped to an F07:{j_to_i}')
            print(f'Expected # of FALSE F07R15 reads:{indiv_tag_jump_count}')
            


Here we see how despite equal read counts between two samples, the differential abundances of extraneous 
tags result in different expected false reads within that sample

Calculating F04R14 Index Jumps
Total F04R14 reads: 6250
Total Reads with F04 and NOT R14:56250
Total Reads with R14 and NOT F04:50000
Expected # of Seqs where F04 jumped to an R14:2.334269923779075
Expected # of Seqs where R14 jumped to an F04:2.626053664251459
Expected # of FALSE F04R14 reads:5

Calculating F10R17 Index Jumps
Total F10R17 reads: 6250
Total Reads with F10 and NOT R17:37985
Total Reads with R15 and NOT F07:43755
Expected # of Seqs where F07 jumped to an R15:1.3794258559504018
Expected # of Seqs where R15 jumped to an F07:1.5764641738498422
Expected # of FALSE F07R15 reads:3


### Summarizing results

From these calculations, we can generate useful summary metrics such as the total number and percent of flase 
reads in our sample pool, as well as a table with expected flase counts per sample which can be used for setting individual filtering levels per-sample to balance data cleaning and read retainment downstream. 

(For help with per-sample fitlering in QIIME2 [click here!](https://github.com/NGabry/Per-Sample-Filtering))

In [9]:
# Creating a dataframe of expected false reads per sample
out_df = pd.DataFrame(indexes_exp_false, columns = ['SampleIndex', 'Expected False Reads'])
out_df

# Printing summary statistics
max_IJR = out_df['Expected False Reads'].max()
false_read_count = out_df['Expected False Reads'].sum()
print(f'Calculated index jump rate: {jump_rate}')
print(f'Total number of false reads: {false_read_count} / {total_read_count}')
print(f'Total percent of false reads: {(false_read_count/total_read_count)*100:.3f}%')
print(f'Maximum number of false reads expected in a single sample: {max_IJR}')

Calculated index jump rate: 0.0003998336716482295
Total number of false reads: 405 / 481749
Total percent of false reads: 0.084%
Maximum number of false reads expected in a single sample: 5


### Comparing to alternative index-jump calculation method

An alternative method of calculating the index jump rate within a sample pool is to identify any reads which contain an index combination that should not exist within a pool, dividing by the total read count to get proportion of false reads, and then multiplying this by two to get our index jump rate due to the fact that this false read could've occured from a jump on the forward or reverse end.

Here, we can replicate this method using our sample map by summing all of our False reads.

In doing this, we see that our jump rate is estimated to be much lower than when utilizing calibrator tags, which could ultimately lead to incorrect source sample tracing in downstream analysis due to improper read filtering. 


In [10]:
# Alternative calculation method 
f_df = df[df['True_False'] == 0]
false_sum = f_df['forward sequence count'].sum()
total_sum = df['forward sequence count'].sum()

print(f'Calculated index jump rate: {(false_sum/total_sum)*2:5f}%')

Calculated index jump rate: 0.000071%
